## training.log 에서 Pareto front 찾은 뒤, full training 후 (chromosome, flops, val_acc_5_epoch, val_acc_full trainig) 이렇게 로그 떨구기

### 1) 테스트해서 로그 제대로 떨어지는 것 확인


### 2) .py 파일로 다시 쓴 뒤 GA, RS 이어서 돌려놓기

### ------------------------------------------------------

# [1] 주어진 log path 에서 training log 읽어와서, 파레토 프론티어 찾기

In [1]:
import numpy as np
import os
import json
import sys
import pandas as pd
sys.path.insert(0,'/root/data/basic_model/')

import matplotlib.pyplot as plt
%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

from utils_kyy.pareto_front import identify_pareto

In [2]:
data_path = '../logs/__New_main_experiment_1_GA_30gen/'

with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])

objs_fitness = []
objs_chromo = []
gen_num = []
for i in range(niter):
    gen_num.extend([i for j in range(npop)])
    fitness_i = [train_log[str(i)][j][1] for j in range(npop)]  # [-val_acc, flops]
    chromo_i = [train_log[str(i)][j][0] for j in range(npop)]  # [-val_acc, flops]
    objs_fitness.append(fitness_i)
    objs_chromo.append(chromo_i)

objs_fitness = np.array(objs_fitness)
epoch = list(range(niter))

objs_fitness[:,:,0]= -1*objs_fitness[:,:,0]  # -val_acc => +val_acc

y1 = objs_fitness[:,:,0].reshape(-1).tolist()  # val_accuracy 는 - 붙어있는채로 사용 => minimize 하는 pareto frontier 찾는 함수 그대로 사용
y2 = objs_fitness[:,:,1].reshape(-1).tolist()
idxs = [i for i in range(len(y1))]
pareto = [0 for i in range(len(y1))]

In [3]:
df = pd.DataFrame({'gen':gen_num,'idx': idxs, 'acc':y1, 'flops': y2})

In [69]:
df.head()

,acc,flops,gen,idx
0,83.75,1.654448e+09,0,0
1,81.71,1.775048e+09,0,1
2,82.22,1.480460e+09,0,2
3,78.12,1.739114e+09,0,3
4,82.56,2.024451e+09,0,4


In [23]:
# 확인
idx = 4
df.iloc[[idx]].values[0]  # acc, floops, gen, idx

array([8.25600000e+01, 2.02445056e+09, 0.00000000e+00, 4.00000000e+00])

In [75]:
## pareto front 찾기
data_30gen_score = df[['acc','flops']].values  # df 전부

# 1) flops 에 - 붙이기 => score 로 만들기
data_30gen_score[:, 1] = -data_30gen_score[:, 1]

# 2) 파레토 프론티어 찾기
pareto_30gen_idx = identify_pareto(data_30gen_score)
pareto_front_30gen = data_30gen_score[pareto_30gen_idx]

# 3) 파레토 프론티어에 있는 크로모좀 리스트 만들기
pareto_chromos = []
for idx in list(pareto_30gen_idx):
    i = int(idx / 20)   # e.g. 33 => 1 * 20 + 13 => 1 gen 의 14번째 => objs_chromo[1][13]  ## 각각 0번째 ~ 19번째 있음
    j = idx - i*20
    temp_chromo = objs_chromo[i][j]
    pareto_chromos.append( temp_chromo )

In [ ]:
###############################################################################################
## 파레토 프론티어에 있는 크로모좀들 풀트레이닝 한 뒤에, acc_full 을 따로 리스트로 저장해놓기
###############################################################################################
acc_full_pt = []
for individual in pareto_chromos:
    # 풀트레이닝 ######################################################################################
    
    
    acc_full_pt.append( ---------  ) ############################################
    

    
########################################################################    
## acc_full list 추가해서 아래와 같은 데이터 프레임 생성 후 저장해놓기
########################################################################
# pareto frontier를 찾은 뒤, full training 까지 해서 완성 할 데이터 프레임
gen_pt = []
idx_pt = list(pareto_30gen_idx)
flos_pt = []
acc_5epoch_pt = []

for idx in list(pareto_30gen_idx):
    df_idx = df.iloc[[idx]].values[0]  # acc, floops, gen, idx

    gen_pt.append(df_idx[2])
    idx_pt.append(df_idx[3])
    flops_pt.append(df_idx[1])
    acc_5epoch_pt.append(df_idx[0])
    

## 데이터 프레임 생성
df_pareto = pd.DataFrame({'gen': gen_pt, 'idx': idx_pt, 'flops': flops_pt, 'acc_5epoch':acc_5epoch_pt, 'acc_full':acc_full_pt})

## 데이터 프레임 저장
save_path = '../logs/Full_Training_191223/pareto_GA.pkl'
df.to_pickle(save_path)  # asdp = pd.read_pickle(load_path)

##################### RS log 에 대해서도 수행하기 #####################################

In [80]:
# [참고] 데이터 프레임 읽어옹기
load_path = '../logs/Full_Training_191223/pareto_GA.pkl'
asdp = pd.read_pickle(load_path)

In [82]:
asdp.shape

(620, 4)